In [1]:
import torch
import yaml

# Local imports
import carl_utils.preprocessing as carl_ppro
import carl_utils.train as carl_train

import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.version.cuda)
#DEVICE = 'cpu'
print(DEVICE)

11.7
cuda:0


In [2]:
total_features = yaml.load(open("carl_features.yaml", 'r'), Loader=yaml.CLoader)
total_features

{'Electron_set': {'set': True,
  'size': 4,
  'subfeatures': ['Electron_pt',
   'Electron_eta',
   'Electron_phi',
   'Electron_mass']},
 'FatJet_set': {'set': True,
  'size': 5,
  'subfeatures': ['FatJet_pt',
   'FatJet_eta',
   'FatJet_phi',
   'FatJet_mass',
   'FatJet_btagCSVV2']},
 'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']},
 'MET_sumEt': {'set': False, 'size': 1, 'subfeatures': ['MET_sumEt']},
 'Muon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass']},
 'Photon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Photon_pt', 'Photon_eta', 'Photon_phi', 'Photon_mass']}}

In [3]:
subfeatures = ["Jet_set", "Electron_set"]#, "Muon_set"]
features = dict(zip(subfeatures, [total_features[feat] for feat in subfeatures]))
features

{'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']},
 'Electron_set': {'set': True,
  'size': 4,
  'subfeatures': ['Electron_pt',
   'Electron_eta',
   'Electron_phi',
   'Electron_mass']}}

In [4]:
weight_features = ["genWeight", "btagWeight_CSVV2"]

## Initiate the datasets and load the data

In [22]:
n_train_events = int(1e6) // 2
n_val_events = int(2e5) // 2

train_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_training_data_new2.root",], features, 0, stop_event=n_train_events)
valid_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_validation_data_new2.root",], features, 0, stop_event=n_val_events)

train_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_training_data_new2.root",], features, 1, stop_event=n_train_events)
valid_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_validation_data_new2.root",], features, 1, stop_event=n_val_events)

100%|██████████| 100000/100000 [00:34<00:00, 2932.68it/s]


## Prepare the data

In [23]:
train_generator_data = carl_ppro.CombinedDataset(train_nominal_dataset, train_PS_var_dataset)
valid_generator_data = carl_ppro.CombinedDataset(valid_nominal_dataset, valid_PS_var_dataset)

## Define the settings of the model and training

In [24]:
hidden1 = 512 // 2
hidden2 = 256 // 2
hidden3 = 128 // 2
classify1 = 1024  // 2
classify2 = 1024 // 2
outputs = 1

model_settings = {
    "features": features,
    "phi": [hidden1, hidden2, hidden3],
    "mlp": [classify1, classify2]
}


training_settings = {
    "optimizer": "Adam",
    "learning_rate": 1e-2,
    "batch_size": 256,
    "n_epochs": 30,
    "patience": 6,
    "device": DEVICE,
    "n_events": n_train_events * 2,
    "saveAs": "models/deepsets_agc_model_new7",
    "return_best_model": True
}

# Perform the training

In [ ]:
model, losses = carl_train.train(model_settings, train_generator_data, valid_generator_data, **training_settings)

Constructing the model
Loading the input data scaling


100%|██████████| 977/977 [00:10<00:00, 92.19it/s] 


Training the model


/home/mdrnevich/miniconda3/envs/carl-for-agc/lib/python3.11/site-packages/torch/onnx/utils.py:825: UserWarning: no signature found for <torch.ScriptMethod object at 0x7f55844b63f0>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


Epoch: 01, Training Loss:   0.6862
           Validation Loss: 0.6828


...

 57%|█████▋    | 17/30 [1:13:21<56:41, 261.63s/it]                 

Epoch: 17, Training Loss:   0.6797
           Validation Loss: 0.6804
Stale epoch


 60%|██████    | 18/30 [1:17:45<52:28, 262.37s/it]                 

Epoch: 18, Training Loss:   0.6796
           Validation Loss: 0.6803
Stale epoch


loss = 0.65158:  60%|█████▉    | 2343/3907 [02:30<01:38, 15.80it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

loss = 0.66589:  24%|██▎       | 927/3907 [00:58<03:09, 15.72it/s] 

In [ ]:
model

In [ ]:
losses